In [1]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import for CRUD module
from crudModule import AnimalShelter



###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = 'SNHU1234'
HOST = 'localhost'
PORT = 27017
DB = 'AAC'
COL = 'animals'


db = AnimalShelter(username)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug test 
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='/assets/Grazioso Salvare Logo.png', 
             style={'width': '300px', 'height': 'auto'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H3("Created by: Jermaine Wiggins"), # unique indentifier 
    html.Hr(),
    html.Div([
    dcc.Dropdown(id = 'dropdown', 
                 options = ['Tracking', 'Water Rescue', 'Mountain Rescue'])]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        selected_columns =[],
        selected_rows = [0], # ensures first row is always selected
        page_action = "native",
        page_current = 0,
        page_size = 10
    ),
    html.Br(),
     html.Hr(),
     html.Div(children = [
            html.H4('Rescue Animals Breed'),
            dcc.Graph(id="graph", style={'display': 'inline-block', 'width': '43%'}), # pie chart and map side by side
            html.Div(id='map-id', style={'display': 'inline-block', 'width': '45%'})
     ])
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will filter based on users choice
@app.callback(
    Output('datatable-id', 'data'),
    Input('dropdown', 'value')
)
def dropdown_output(value):
    # Read all data first fresh from database
    df = pd.DataFrame.from_records(db.read({}))

    # if no filter selected, return all data after dropping '_id' to prevent JSON error
    if not value:
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
        return df.to_dict('records')

    # Build filter list for MongoDB query with $or
    filters = []

    #types of dogs suitable for rescue type

    tracking = {"animal_type": "Dog", 
                "breed": { "$in": [ "Doberman Pinscher", "German Shepherd", "Golden Retriever","Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20,"$lte": 300}
               }

    waterRescue = {"animal_type": "Dog", 
                "breed": { "$in": [ "Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26,"$lte": 156}
               }

    mountainRescue = {"animal_type": "Dog", 
                "breed": { "$in": [ "German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky",
                                    "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26,"$lte": 156}
               }
  

    # check for click from drop down
    if 'Tracking' in value:
        filters.append(tracking)
    elif 'Water Rescue' in value:
        filters.append(waterRescue)
    elif 'Mountain Rescue' in value:
        filters.append(mountainRescue) 
    
    # checks if filters empty
    if filters:
        query = {"$or": filters} # creates single or compound query
    else:
        query = {}

    # Query the database with combined filters
    df = pd.DataFrame.from_records(db.read(query))

    # Drop '_id' to prevent JSON error
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # update chart
    return df.to_dict('records')
   


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
   # error handling
    try:
        # Convert the datatable view data to a DataFrame
        dff = pd.DataFrame.from_dict(viewData)
        # Checks if dff empty
        if dff.empty:
            return []

        # Because we only allow single row selection, pick first row if nothing selected
        if not index:
            row = 0
        else:
            row = index[0]

    # Return a leaflet map centered at Austin, TX
        return [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    # Marker from lat/long columns 13 and 14
                    dl.Marker(
                        position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                        children=[
                            dl.Tooltip(dff.iloc[row, 4]),     
                            dl.Popup([
                                html.H1("Animal Name"), 
                                html.P(dff.iloc[row, 9])
                            ])
                        ]
                    )
                ]
             )
        ]
    except Exception as e:
        print(f"Error in update_map: {e}")
        return[]

@app.callback(
    Output("graph", "figure"),
    Input('datatable-id', "derived_virtual_data"),
     )

def generate_chart(viewData):
    # check if view data empty
    if not viewData:
        return {}
    # Convert the datatable view data to a DataFrame    
    dff = pd.DataFrame.from_dict(viewData) 
    
    # counts how many times a breed appears
    counts = dff['breed'].value_counts()
    
    # gets total amount of all breeds combined
    total = counts.sum()

    # create threshold for displayed data
    threshold = total * 0.01

    # find breeds less than 1%
    breeds_greater_than_1_percent = counts[counts >= threshold].copy()
    breeds_less_than_1_percent = counts[counts < threshold].sum()

    # add other category and set to breeds_less_than_1_percent
    if breeds_less_than_1_percent > 0:
       breeds_greater_than_1_percent['Other'] = breeds_less_than_1_percent

    # create new datframe and rename columns
    new_dff = breeds_greater_than_1_percent.reset_index()
    new_dff.columns = ['breed', 'count']
    fig = px.pie(new_dff, values='count', names='breed', hole=.3)
    return fig


app.run(debug=True)